<div style="background:#FFFFAA">
<img src="https://www.google.com/url?sa=i&rct=j&q=&esrc=s&source=images&cd=&cad=rja&uact=8&ved=2ahUKEwiDjJbig-beAhVJbBoKHWsOAB4QjRx6BAgBEAU&url=https%3A%2F%2Fwww.analyticsvidhya.com%2Fblog%2F2014%2F04%2Fsurvival-analysis-model-you%2F&psig=AOvVaw1_olVE3Ztrjfv_qSvtzLsW&ust=1542908372712541", width=150, ALIGN="left", border=20>
<h1>Sample Starting Kit </h1>
    <span style="color:red"> <h1> Keep all code below **yellow** text blocks, </h1> </span>
    <span style="color:red"> <h1> but check that it works (you can ADD not remove). </h1> </span>
    <span style="color:red"> <h1> Customize Section (1) and modify the explanations.</h1> </span>

<br>This code was tested with <br>
Python 3.6.6 |Anaconda custom (64-bit)| (default, Jun 28 2018, 11:07:29) (https://anaconda.org/)<br>
<i> Adapted for Chalab by Isabelle Guyon from original code of Balázs Kégl</i> <br>
<a href="http://www.datascience-paris-saclay.fr">Paris Saclay Center for Data Science (CDS)</a>
</center>
<p>
ALL INFORMATION, SOFTWARE, DOCUMENTATION, AND DATA ARE PROVIDED "AS-IS". The CDS, CHALEARN, AND/OR OTHER ORGANIZERS OR CODE AUTHORS DISCLAIM ANY EXPRESSED OR IMPLIED WARRANTIES, INCLUDING, BUT NOT LIMITED TO, THE IMPLIED WARRANTIES OF MERCHANTABILITY AND FITNESS FOR ANY PARTICULAR PURPOSE, AND THE WARRANTY OF NON-INFRIGEMENT OF ANY THIRD PARTY'S INTELLECTUAL PROPERTY RIGHTS. IN NO EVENT SHALL AUTHORS AND ORGANIZERS BE LIABLE FOR ANY SPECIAL, 
INDIRECT OR CONSEQUENTIAL DAMAGES OR ANY DAMAGES WHATSOEVER ARISING OUT OF OR IN CONNECTION WITH THE USE OR PERFORMANCE OF SOFTWARE, DOCUMENTS, MATERIALS, PUBLICATIONS, OR INFORMATION MADE AVAILABLE FOR THE CHALLENGE. 
</div>

<div style="background:#FFFFAA">
    <h2>Introduction </h2>
    <p>
    <span style="color:red"> Here write here a short description of your problem. </span>
     <br>
In this challenge we try to tackle a survival analysis problem from a machine learning perspective. The goal of survival analysis is to predict the expected duration before one event occur, usually death for living being but also by extension failure in mechanical systems. In our case we try to estimate the time of death of a population
 <br>
    <span style="color:red"> Keep the next block. </span>
</div>

In [5]:
model_dir = 'sample_code_submission/'                        # Change the model to a better one once you have one!
#model_dir = '../FILES/pretty_good_sample_code_submission/'
result_dir = 'sample_result_submission/' 
problem_dir = 'ingestion_program/'  
score_dir = 'scoring_program/'
from sys import path; path.append(model_dir); path.append(problem_dir); path.append(score_dir); 
%matplotlib inline
# Uncomment the next lines to auto-reload libraries (this causes some problem with pickles in Python 3)
%load_ext autoreload
%autoreload 2
import seaborn as sns; sns.set()
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


<div style="background:#FFFFAA">
    <h1> Step 1: Exploratory data analysis </h1>
<p>
We provide sample_data with the starting kit, but to prepare your submission, you must fetch the public_data from the challenge website and point to it.
    <br>
    <span style="color:red"> Just change the data name in the block below. In the rest of the section, replace the sample plots by anything you want. </span>
</div>

In [8]:
data_dir = 'sample_data'              # Change this to the directory where you put the input data
#data_dir = '../FILES/iris/'          # The sample_data directory should contain only a very small subset of the data
data_name = 'Mortality'
!ls $data_dir*

Mortality_feat.name      Mortality_test.data      Mortality_train.solution
Mortality_label.name     Mortality_test.solution  Mortality_valid.data
Mortality_public.info    Mortality_train.data     Mortality_valid.solution


For convenience, we load the data as a "pandas" data frame, so we can use "pandas" and "seaborn" built in functions to explore the data.

In [19]:
from data_io import read_as_df
data = read_as_df(data_dir  + '/' + data_name)                # The data are loaded as a Pandas Data Frame

Reading sample_data/Mortality_train from AutoML format
Number of examples = 19297
Number of features = 11


In [20]:
data.head()

,mortstat,age,sbp,dbp,A1c,bmi,gender_male,ethnicity_NonHispanicBlack,ethnicity_NonHispanicWhite,ethnicity_Other,ethnicity_OtherHispanic
0,0.0,50,124.666667,70.000000,5.3,30.74,0,0,0,0,1
1,0.0,49,119.333333,72.666667,5.1,23.80,1,0,1,0,0
2,0.0,44,134.666667,78.666667,5.9,27.65,1,0,1,0,0
3,0.0,50,119.000000,87.000000,4.9,24.77,0,0,1,0,0
4,0.0,32,104.666667,72.000000,4.8,26.45,1,0,1,0,0


In [21]:
data.describe() 

,mortstat,age,sbp,dbp,A1c,bmi,gender_male,ethnicity_NonHispanicBlack,ethnicity_NonHispanicWhite,ethnicity_Other,ethnicity_OtherHispanic
count,19297.000000,19297.000000,19297.000000,19297.000000,19297.000000,19297.000000,19297.000000,19297.000000,19297.000000,19297.000000,19297.000000
mean,0.097580,46.531793,124.198995,69.418321,5.569762,28.192723,0.483702,0.201275,0.479194,0.037830,0.057315
std,0.296754,19.853825,20.062107,13.795536,1.020326,6.261523,0.499747,0.400963,0.499580,0.190789,0.232449
min,0.000000,18.000000,72.666667,0.000000,2.000000,12.040000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,29.000000,110.666667,62.000000,5.100000,23.820000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,45.000000,120.000000,70.000000,5.400000,27.260000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,0.000000,63.000000,134.000000,77.333333,5.700000,31.470000,1.000000,0.000000,1.000000,0.000000,0.000000
max,1.000000,85.000000,270.000000,124.000000,18.800000,76.070000,1.000000,1.000000,1.000000,1.000000,1.000000


## AJOUTER DES STATS SUR LES DONNEES CENSUREES COMME ALEX ET FAIRE QUELQUES COURBRES KM SI POSSIBLE 

<div style="background:#FFFFAA">
<h1>Step 2: Building a predictive model</h1>
</div>

<div style="background:#FFFFAA">
    <h2>Loading data with DataManager</h2>
    <p>
We reload the data with the AutoML DataManager class because this is more convenient:
   <br>     <span style="color:red"> Keep this, it illustrates how data in AutoML formal are loaded by the ingestion program </span>
</div>

In [23]:
from data_manager import DataManager
D = DataManager(data_name, data_dir, replace_missing=True)
print(D)

Info file found : /Users/nour/Desktop/AIC/mini_projet/projet_AIC_Survival/sample_data/Mortality_public.info
DataManager : Mortality
info:
	usage = Continuous NHANES Mortality data
	name = mortality
	task = regression
	target_type = Numerical
	feat_type = Numerical
	metric = mse_metric
	time_budget = 1200
	feat_num = 11
	target_num = 1
	label_num = 2
	train_num = 19297
	valid_num = 2171
	test_num = 2412
	has_categorical = 0
	has_missing = 0
	is_sparse = 0
	format = dense
data:
	X_train = array(19297, 11)
	Y_train = array(19297,)
	X_valid = array(2413, 11)
	Y_valid = array(2413,)
	X_test = array(2412, 11)
	Y_test = array(2412,)
feat_type:	array(11,)
feat_idx:	array(0,)



<div style="background:#FFFFAA">
    <h2>Training a predictive model</h2>
    <p>
We provide an example of predictive model (for classification or regression) in the `sample_code_submission/` directory. It is a quite stupid model: it makes constant predictions. Replace it with your own model.
    </div>

In [24]:
from data_io import write
from model import model
# Uncomment the next line to show the code of the model
#??model 

<div style="background:#FFFFAA">
an instance of the model (run the constructor) and attempt to reload a previously saved version from `sample_code_submission/`:
    
</div>

In [25]:
M = model()
trained_model_name = model_dir + data_name
# Uncomment the next line to re-load an already trained model
#M = M.load(trained_model_name)                  

<div style="background:#FFFFAA">
    Train the model (unless you reloaded a trained model) and make predictions. 
</div>

In [26]:
if not(M.is_trained):
    X_train = D.data['X_train']
    Y_train = D.data['Y_train']
    M.fit(X_train, Y_train)                     

Y_hat_train = M.predict(D.data['X_train']) # Optional, not really needed to test on taining examples
Y_hat_valid = M.predict(D.data['X_valid'])
Y_hat_test = M.predict(D.data['X_test'])

FIT: dim(X)= [19297, 11]
FIT: dim(y)= [19297, 1]
PREDICT: dim(X)= [19297, 11]
PREDICT: dim(y)= [19297, 1]
PREDICT: dim(X)= [2413, 11]
PREDICT: dim(y)= [2413, 1]
PREDICT: dim(X)= [2412, 11]
PREDICT: dim(y)= [2412, 1]


<div style="background:#FFFFAA">
    <b> Save the trained model </b> (will be ready to reload next time around) and save the prediction results. IMPORTANT: if you save the trained model, it will be bundled with your sample code submission. Therefore your model will NOT be retrained on the challenge platform. Remove the pickle from the submission if you want the model to be retrained on the platform.
</div>

In [27]:
M.save(trained_model_name)                 
result_name = result_dir + data_name
from data_io import write
write(result_name + '_train.predict', Y_hat_train)
write(result_name + '_valid.predict', Y_hat_valid)
write(result_name + '_test.predict', Y_hat_test)
!ls $result_name*

sample_result_submission/Mortality_test.predict
sample_result_submission/Mortality_train.predict
sample_result_submission/Mortality_valid.predict


<div style="background:#FFFFAA">
    <h2>Scoring the results</h2>
    <h3>Load the challenge metric</h3>
    <p>
<b>The metric chosen for your challenge</b> is identified in the "metric.txt" file found in the `scoring_function/` directory. The function "get_metric" searches first for a metric having that name in my_metric.py, then in libscores.py, then in sklearn.metric.
    </div>

In [28]:
from libscores import get_metric
metric_name, scoring_function = get_metric()
print('Using scoring metric:', metric_name)
# Uncomment the next line to display the code of the scoring metric
#??scoring_function

Using scoring metric: mse_metric


<div style="background:#FFFFAA">
    <h3> Training performance </h3>
    <p>
The participants normally posess target values (labels) only for training examples (except for the sample data). We compute with the `example` metric the training score, which should be zero for perfect predictions.
        </div>

In [29]:
print('Training score for the', metric_name, 'metric = %5.4f' % scoring_function(Y_train, Y_hat_train))
print('Ideal score for the', metric_name, 'metric = %5.4f' % scoring_function(Y_train, Y_train))

Training score for the mse_metric metric = 0.0000
Ideal score for the mse_metric metric = 0.0000


You can add here other scores and result visualization.

In [30]:
from sklearn.metrics import confusion_matrix
confusion_matrix(Y_train, Y_hat_train)

array([[  4,   0,   0, ...,   0,   0,   0],
       [  0,  15,   0, ...,   0,   0,   0],
       [  0,   0,   8, ...,   0,   0,   0],
       ..., 
       [  0,   0,   0, ..., 140,   0,   0],
       [  0,   0,   0, ...,   0, 154,   0],
       [  0,   0,   0, ...,   0,   0,  64]])

<div style="background:#FFFFAA">
    <h3>Cross-validation performance</h3>
    <p>
The participants do not have access to the labels Y_valid and Y_test to self-assess their validation and test performances. But training performance is not a good prediction of validation or test performance. Using cross-validation, the training data is split into multiple training/test folds, which allows participants to self-assess their model during development. The average CV result and 95% confidence interval is displayed.
   </div>

In [31]:
from sklearn.metrics import make_scorer
from sklearn.model_selection import cross_val_score
scores = cross_val_score(M, X_train, Y_train, cv=5, scoring=make_scorer(scoring_function))
print('\nCV score (95 perc. CI): %0.2f (+/- %0.2f)' % (scores.mean(), scores.std() * 2))

FIT: dim(X)= [15437, 11]
FIT: dim(y)= [15437, 1]
PREDICT: dim(X)= [3860, 11]
PREDICT: dim(y)= [3860, 1]
FIT: dim(X)= [15437, 11]
FIT: dim(y)= [15437, 1]
PREDICT: dim(X)= [3860, 11]
PREDICT: dim(y)= [3860, 1]
FIT: dim(X)= [15438, 11]
FIT: dim(y)= [15438, 1]
PREDICT: dim(X)= [3859, 11]
PREDICT: dim(y)= [3859, 1]
FIT: dim(X)= [15438, 11]
FIT: dim(y)= [15438, 1]
PREDICT: dim(X)= [3859, 11]
PREDICT: dim(y)= [3859, 1]
FIT: dim(X)= [15438, 11]
FIT: dim(y)= [15438, 1]
PREDICT: dim(X)= [3859, 11]
PREDICT: dim(y)= [3859, 1]

CV score (95 perc. CI): 2283.45 (+/- 60.25)


<div style="background:#FFFFAA">
<h1> Step 3: Making a submission </h1> 

<h2> Unit testing </h2> 

It is <b><span style="color:red">important that you test your submission files before submitting them</span></b>. All you have to do to make a submission is modify the file <code>model.py</code> in the <code>sample_code_submission/</code> directory, then run this test to make sure everything works fine. This is the actual program that will be run on the server to test your submission. 
<br>
Keep the sample code simple.
</div>

In [32]:
!source activate python3; python $problem_dir/ingestion.py $data_dir $result_dir $problem_dir $model_dir

Could not find conda environment: python3
You can list all discoverable environments with `conda info --envs`.

Using input_dir: /Users/nour/Desktop/AIC/mini_projet/projet_AIC_Survival/sample_data
Using output_dir: /Users/nour/Desktop/AIC/mini_projet/projet_AIC_Survival/sample_result_submission
Using program_dir: /Users/nour/Desktop/AIC/mini_projet/projet_AIC_Survival/ingestion_program
Using submission_dir: /Users/nour/Desktop/AIC/mini_projet/projet_AIC_Survival/sample_code_submission

========== Ingestion program version 6 ==========

************************************************
******** Processing dataset Mortality ********
************************************************
========= Reading and converting data ==========
Info file found : /Users/nour/Desktop/AIC/mini_projet/projet_AIC_Survival/sample_data/Mortality_public.info
========= Reading /Users/nour/Desktop/AIC/mini_projet/projet_AIC_Survival/sample_data/Mortality_feat.type
[+] Success in  0.00 sec
========= Reading /Users/

<div style="background:#FFFFAA">
Also test the scoring program:
    </div>

In [33]:
scoring_output_dir = '../scoring_output_dir'
!source activate python3; python $score_dir/score.py $data_dir $result_dir $scoring_output_dir

Could not find conda environment: python3
You can list all discoverable environments with `conda info --envs`.

======= Set 1 (Mortality_test): mse_metric(set1_score)=2280.660862354892 =======
======= Set 2 (Mortality_train): mse_metric(set2_score)=0.000000000000 =======
======= Set 3 (Mortality_valid): mse_metric(set3_score)=2298.426025694157 =======


<div style="background:#FFFFAA">
    <h1> Preparing the submission </h1>

Zip the contents of `sample_code_submission/` (without the directory), or download the challenge public_data and run the command in the previous cell, after replacing sample_data by public_data.
Then zip the contents of `sample_result_submission/` (without the directory).
<b><span style="color:red">Do NOT zip the data with your submissions</span></b>.

In [34]:
import datetime 
from data_io import zipdir
the_date = datetime.datetime.now().strftime("%y-%m-%d-%H-%M")
sample_code_submission = '../sample_code_submission_' + the_date + '.zip'
sample_result_submission = '../sample_result_submission_' + the_date + '.zip'
zipdir(sample_code_submission, model_dir)
zipdir(sample_result_submission, result_dir)
print("Submit one of these files:\n" + sample_code_submission + "\n" + sample_result_submission)

Submit one of these files:
../sample_code_submission_18-11-21-21-37.zip
../sample_result_submission_18-11-21-21-37.zip
